Exercício: usando um modelo treinado em novos dados
Na Unidade 3, criamos um modelo básico que nos permite encontrar a relação entre o tamanho do arreio de um cão e o tamanho da bota. Mostramos como esse modelo poderia ser usado para fazer uma previsão sobre um novo cão nunca visto.

É comum construir, treinar e usar um modelo enquanto estamos aprendendo sobre aprendizado de máquina, mas no mundo real não queremos treinar o modelo toda vez que queremos fazer uma previsão.

Considere nosso cenário de loja de equipamentos para cães de avalanche:

Queremos treinar o modelo apenas uma vez e, em seguida, carregar esse modelo no servidor que executa nossa loja online.
Embora o modelo seja treinado em um conjunto de dados que baixamos da Internet, na verdade queremos usá -lo para estimar o tamanho da inicialização dos cães de nossos clientes que não estão nesse conjunto de dados!
Como podemos fazer isso?

Aqui iremos:

Crie um modelo básico
Salve-o em disco
Carregue-o do disco
Use-o para fazer previsões sobre um cão que não estava no conjunto de dados de treinamento.
Carregar o conjunto de dados
Vamos começar abrindo o conjunto de dados do arquivo

In [1]:
import pandas
!pip install statsmodels
!python -m wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!python -m wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-boot-harness.csv

# Load a file containing dog's boot and harness sizes
data = pandas.read_csv('doggy-boot-harness.csv')

# Print the first few rows
data.head()


Saved under graphing (2).py

Saved under doggy-boot-harness (2).csv


,boot_size,harness_size,sex,age_years
0,39,58,male,12.0
1,38,58,male,9.6
2,37,52,female,8.6
3,39,58,male,10.2
4,38,57,male,7.8


Criar e treinar um modelo
Como fizemos antes, vamos criar um modelo simples de Regressão Linear e treiná-lo em nosso conjunto de dados.

In [2]:
import statsmodels.formula.api as smf

# Fit a simple model that finds a linear relationship
# between boot size and harness size, which we can use later
# to predict a dog's boot size, given their harness size
model = smf.ols(formula = "boot_size ~ harness_size", data = data).fit()

print("Model trained!")

Model trained!


Salvando e carregando um modelo
Nosso modelo está pronto para uso, mas ainda não precisamos dele. Vamos salvá-lo em disco.

In [3]:
import joblib

model_filename = './avalanche_dog_boot_model.pkl'
joblib.dump(model, model_filename)

print("Model saved!")

Model saved!


Carregar nosso modelo é tão fácil quanto:

In [4]:
model_loaded = joblib.load(model_filename)

print("We have loaded a model with the following parameters:")
print(model_loaded.params)

We have loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64


Juntar as peças
Em nosso site, queremos pegar o arnês do cachorro do nosso cliente e calcular o tamanho da bota do cachorro usando o modelo que já treinamos.

Vamos juntar tudo aqui para fazer uma função que carrega o modelo do disco, e então o usa para prever a altura do tamanho da inicialização do cachorro do nosso cliente.

In [5]:
# Let's write a function that loads and uses our model
def load_model_and_predict(harness_size):
    '''
    This function loads a pretrained model. It uses the model
    with the customer's dog's harness size to predict the size of
    boots that will fit that dog.

    harness_size: The dog harness size, in cm 
    '''

    # Load the model from file and print basic information about it
    loaded_model = joblib.load(model_filename)

    print("We have loaded a model with the following parameters:")
    print(loaded_model.params)

    # Prepare data for the model
    inputs = {"harness_size":[harness_size]} 

    # Use the model to make a prediction
    predicted_boot_size = loaded_model.predict(inputs)[0]

    return predicted_boot_size

# Practice using our model
predicted_boot_size = load_model_and_predict(45)

print("Predicted dog boot size:", predicted_boot_size)

We have loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64
Predicted dog boot size: 32.08575356590473


Uso no mundo real
Conseguimos - podemos prever o tamanho da bota de um cão de avalanche, com base no tamanho de seu arnês. Nosso último passo é usar isso para alertar as pessoas se elas podem estar comprando as botas caninas do tamanho errado.

Como exemplo, faremos uma função que aceita o tamanho do arnês, o tamanho das botas selecionadas e retorna uma mensagem para o cliente. Iríamos integrar esta função em nossa loja online.

In [6]:
def check_size_of_boots(selected_harness_size, selected_boot_size):
    '''
    Calculates whether the customer has chosen a pair of doggy boots that 
    are a sensible size. This works by estimating the dog's actual boot 
    size from their harness size.

    This returns a message for the customer that should be shown before
    they complete their payment 

    selected_harness_size: The size of the harness the customer wants to buy
    selected_boot_size: The size of the doggy boots the customer wants to buy
    '''

    # Estimate the customer's dog's boot size
    estimated_boot_size = load_model_and_predict(selected_harness_size)

    # Round to the nearest whole number because we don't sell partial sizes
    estimated_boot_size = int(round(estimated_boot_size))

    # Check if the boot size selected is appropriate
    if selected_boot_size == estimated_boot_size:
        # The selected boots are probably OK
        return f"Great choice! We think these boots will fit your avalanche dog well."

    if selected_boot_size < estimated_boot_size:
        # Selected boots might be too small 
        return "The boots you have selected might be TOO SMALL for a dog as "\
               f"big as yours. We recommend a doggy boots size of {estimated_boot_size}."

    if selected_boot_size > estimated_boot_size:
        # Selected boots might be too big 
        return "The boots you have selected might be TOO BIG for a dog as "\
               f"small as yours. We recommend a doggy boots size of {estimated_boot_size}."
    

# Practice using our new warning system
check_size_of_boots(selected_harness_size=55, selected_boot_size=39)

We have loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64


'The boots you have selected might be TOO BIG for a dog as small as yours. We recommend a doggy boots size of 38.'

Altere selected_harness_sizee selected_boot_sizeno exemplo acima e execute novamente para ver isso em ação.

Resumo
Bem feito! Criamos um sistema que pode prever se os clientes estão comprando botas caninas que podem não servir para o seu cão de avalanche, com base apenas no tamanho do arnês que estão comprando.

Neste exercício, praticamos:

Criando modelos básicos
Treinar e depois salvá-los em disco
Carregando-os do disco
Fazendo previsões com eles usando novos conjuntos de dados